<a href="https://colab.research.google.com/github/yoouza/bigcontest/blob/master/data_analysis/News_Classification_KoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1
!pip install tensorflow

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x38c6a000 @  0x7ff881ea6615 0x591f47 0x4cc229 0x4cc38b 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x509918 0x50a64d
     |████████████████████████████████| 846.0MB 22kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 12.1MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588528 sha256=6e62e7f0b51e53e329eb4d0b177bd9ffdc5ec186b1da523904bd20ca6aebca0c
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.0MB 14.8

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git

  Cloning https://****@github.com/SKTBrain/KoBERT.git to /tmp/pip-req-build-dapmwdlj
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-dapmwdlj
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12825 sha256=e9fbfb45e32e2fe570f9b58f03f854683837c50c3d5e0145658912ec4f2d3fd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-1z7l2uhp/wheels/43/7b/50/91bc4d7bebb30b62f31a30a537787eb512f27506909162ba14
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, tqdm_notebook

In [4]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
#GPU 사용 시
device = torch.device("cuda:0")

# **SKT의 KoBERT 이용**

## **데이터 불러오기**

In [6]:
data_path = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/전처리한 데이터/news_data.csv'

In [7]:
news_data = pd.read_csv(data_path, encoding='utf-8', index_col=0)
news_data.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,date,content,label
0,2019-02-07,이귀원 특파원 = 미국 노동부는 지난주 실업수당 청구 건수가 23만4천 건으로 전...,10
1,2019-02-07,실업수당 청구가 줄었다는 것은 그만큼 고용 사정이 좋아졌다는 의미다.,10
2,2019-02-07,미 연방정부의 셧다운 여파 등으로 약 1년 반 만에 최대를 기록했던 2주 전보다 크...,10
3,2019-02-07,다만 블룸버그 전망치보다는 많았다.,10
4,2019-02-07,1972년 故 김규남 의원 애도 글 발표했다가 징역형…고법 억울한 누명으로 고통 ...,10


In [8]:
len(news_data)

1028819

In [9]:
news_data = news_data[news_data['label']!=10]
news_data = news_data.reset_index()
news_data = news_data.drop(['index'], axis=1)
news_data.head()

,date,content,label
0,2019-02-09,김주형 기자 = 9일 오후 서울 광화문 광장에서 열린 비정규직 노동자 고 김용균 ...,1
1,2019-02-09,도널드 트럼프 대통령은 8일 2차 북미정상회담이 오는 27일과 28일 베트남 하노...,1
2,2019-02-09,트럼프 대통령은 이날 트위터를 통해 우리측 대표가 생산적인 만남을 마치고 북한을 막...,1
3,2019-02-09,자세한 내용은 영상으로 확인하시죠.,1
4,2019-02-09,김주형 기자 = 9일 오후 서울 광화문 광장에서 비정규직 노동자 고 김용균 씨의 ...,1


In [10]:
# 학습, 테스트 데이터 분리
train_data = news_data.sample(frac=0.8, random_state=2020)
test_data = news_data.drop(train_data.index)
len(train_data), len(test_data)

(327510, 81877)

In [11]:
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.drop(['index'], axis=1)
test_data = test_data.dropna()
test_data = test_data.reset_index()
test_data = test_data.drop(['index'], axis=1)
len(train_data), len(test_data)

(327510, 81877)

In [12]:
dataset_train = [] # 라벨은 0부터 순서대로 입력해야함
dataset_test = []
for i in tqdm(range(len(train_data[:50]))):
    dataset_train.append([train_data['content'][i], int(train_data['label'][i])]) # 해당 리스트 형태를 맞춰야 학습 가능
for i in tqdm(range(len(test_data[:50]))):
    dataset_test.append([test_data['content'][i], int(test_data['label'][i])])

dataset_test[:5], dataset_train[:5]

100%|██████████| 50/50 [00:00<00:00, 8883.23it/s]


([['전자발찌   김소연 기자 = 전자발찌의 휴대용 위치추적장치를 버리고 달아났던 성범죄자가 4시간 40분만에 붙잡혔다.', 1],
  ['또한 더슬립 쇼룸에 상주한 수면 전문가를 통해 각 매트리스 브랜드별 상세 스펙에 대한 설명을 들을 수 있다.', 1],
  [' 류수현 기자 = 홍역 유행지역인 경기도 안산에서 9일 확진 환자 1명이 추가 발생했다.', 1],
  ['홍역은 홍역 바이러스에 의한 급성 발진성 질환으로 전염성이 매우 높으며 기침 또는 재채기를 통해 공기로 전파된다.', 1],
  ['2019.2.9   김소연 기자 = 9일 오전 4시 27분께 충남 보령시 오천면 회덕항 서쪽 3㎞ 인근 해상에서 740t급 예인선 A호의 기관실 일부가 침수됐다.',
   1]],
 [['김 대법원장은 2017년 사법부 내부에서 촉발된 사법행정권 남용 의혹은 대법원의 3차에 걸친 자체조사 및 검찰의 수사에 의한 진상규명의 과정을 거쳤다.',
   1],
  ['역외시장 원/달러 환율은 1,125.6원으로 0.9원 상승했다.', 0],
  ['이날 위안화 값도 큰 폭으로 떨어졌다.', 0],
  ['맬패스 차관은 도널드 트럼프 미국 대통령에 대한 충성파로 WB의 역할 확대를 주장해왔다.', 1],
  ['같은 기간 제1회 선거 당시 금품선거사범 숫자에 비해 늘었다.', 1]])

## **데이터 토큰화**

In [13]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [14]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
# 파라미터 세팅
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1  # 테스트용이기에 에폭은 1만 지정하겠습니다.
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [17]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## **모델링**

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2, # 해당 부분 파라미터 조정으로 다중 분류 가능
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total) # Warm-Up Step 추가

In [ ]:
def calc_accuracy(X,Y): # 정확도 계산 함수
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## **모델 학습**

In [ ]:
PATH = "/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/분석 모델/model_checkpoint/숙박업/{}_epochs/".format(num_epochs) # 모델 저장 경로 설정

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    # 모델 학습 : model.train()을 지정해줘야 Fine Tunning이 이뤄짐
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # 모델 평가 : model.eval()을 지정해야 Fine Tunning을 멈추고 평가를 시작함
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # 에폭마다 체크포인트 저장
    torch.save(model, PATH + 'model.pt')  # 전체 모델 저장
    torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
    torch.save({
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict()
                }, PATH + 'all.tar')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 1 batch id 1 loss 0.738755464553833 train acc 0.46875
epoch 1 batch id 201 loss 0.5981937646865845 train acc 0.5914956467661692


KeyboardInterrupt: ignored

## **모델링 결과 시각화**

In [22]:
load_model = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/분석 모델/model_checkpoint/숙박업/1_epochs/all.tar'
state_dict = '/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/분석 모델/model_checkpoint/숙박업/1_epochs/model_state_dict.pt'

In [30]:
#model = torch.describe_model()
checkpoint = torch.load(load_model)
model.load_state_dict(torch.load(state_dict))

<All keys matched successfully>

In [29]:
model.load_state_dict?

In [20]:
state_dict

'/content/drive/My Drive/Bigcon/2020빅콘테스트 문제데이터(혁신아이디어분야)/분석 모델/model_checkpoint/숙박업/model_state_dict.pt'